In [ ]:
# TODO: later export script to py file

In [4]:
%%bash
# open output in text editor
cat /homes/fy105/pytorch-mums/dirs.txt

bash: /import/linux/miniconda/3/4.7.12/lib/libtinfo.so.6: no version information available (required by bash)


/import/c4dm-datasets/MUMS
/import/c4dm-datasets/MUMS/DVD 2
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/CELESTA
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN/Pedals_BarPlen_Reeds
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN/SymPlenum_56
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN/Posaune pedals
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN/Prinzipal 2
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN/Nasat
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN/Crumhorn
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN/Hauptwerk All_27
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN/Dulzian
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN/SoloCornet_29
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN/Scharf4_28
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN/Prinzipal4_28
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN/Koppelflote_Positiv
/import/c4dm-datasets/MUMS/DVD 2/KEYBOARDS/ORGAN/

In [1]:
from directory_tree import DisplayTree

path = '/homes/fy105/pytorch-mums/dirs.txt'

# analysis of subdirs in MUMS - quest for data taxonomy
    # https://docs.python.org/3/library/os.path.html#os.path.relpath
    # https://pypi.org/project/directory-tree/